##Modern BERT

Train and test on ISHate dataset, then evaluate with the microaggressions dataset.

In [ ]:
import torch
print(torch.cuda.is_available())  # should be True
print(torch.cuda.get_device_name(0))  # should say T4

True
Tesla T4


In [ ]:
!pip install -q transformers
!pip install -q torchinfo
!pip install -U -q datasets
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import numpy as np

import transformers
import evaluate

from torchinfo import summary

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

splits = {
    'train': 'ishate_train.parquet.gzip',
    'validation': 'ishate_dev.parquet.gzip',
    'test': 'ishate_test.parquet.gzip'
}

df_train = pd.read_parquet("hf://datasets/BenjaminOcampo/ISHate/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/BenjaminOcampo/ISHate/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/BenjaminOcampo/ISHate/" + splits["test"])
max_sequence_length = 128

In [ ]:
# create DatasetDict
ishate_dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "validation": Dataset.from_pandas(df_val),
    "test": Dataset.from_pandas(df_test)
})

In [ ]:
#Encode labels
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(ishate_dataset['train']['hateful_layer'])
y_val_encoded = label_encoder.transform(ishate_dataset['validation']['hateful_layer'])
y_test_encoded = label_encoder.transform(ishate_dataset['test']['hateful_layer'])
ishate_train_data = ishate_dataset['train'].add_column('label', y_train_encoded.tolist())
ishate_val_data = ishate_dataset['validation'].add_column('label', y_val_encoded.tolist())
ishate_test_data = ishate_dataset['test'].add_column('label', y_test_encoded.tolist())


In [ ]:
#sanity check

df_train_preview = pd.DataFrame(ishate_train_data[:10])
df_train_preview


,message_id,text,cleaned_text,source,hateful_layer,implicit_layer,subtlety_layer,implicit_props_layer,aug_method,target,label
0,14430526_4,Another songwriter encouraged many more people...,another songwriter encouraged many more people...,wsf,Non-HS,None,None,None,orig,Non-Specified or General Targets,1
1,30428464_2,The German men sound so sexy .,the german men sound so sexy .,wsf,Non-HS,None,None,None,orig,Women,1
2,500,Like a septic tank.,like a septic tank .,youtube,Non-HS,None,None,None,orig,People with Disabilities,1
3,1942,UK Pensioner Faces 350 Lashes In Saudi Arabia ...,uk pensioner faces lashes in saudi arabia why ...,hateval,Non-HS,None,None,None,orig,Nationalities/Regions,1
4,645,#Burundi—Nkurunziza’s envoy to the UN now reco...,#burundi nkurunziza s envoy to the un now re...,hateval,Non-HS,None,None,None,orig,Immigrants and Refugees,1
5,1766,fuk that I burned a qu'ran and pissed on it...,fuk that i burned a qu ran and pissed on it .,youtube,HS,Explicit HS,Non-Subtle,None,orig,Religious Groups,0
6,8614,You left me in the dark for some bitch that ha...,you left me in the dark for some bitch that ha...,hateval,Non-HS,None,None,None,orig,Women,1
7,18527.0,An Illegal Alien took his own life because he ...,an illegal alien took his own life because he ...,hateval,HS,Explicit HS,Non-Subtle,None,orig,Immigrants and Refugees,0
8,3382.0,@MailOnline Bloody Germany who needs Germany w...,mailonline bloody germany who needs germany we...,hateval,HS,Explicit HS,Non-Subtle,None,orig,Immigrants and Refugees,0
9,1640,"Immigrants are not welcome here, and should no...","immigrants are not welcome here , and should n...",mconan,HS,Explicit HS,Non-Subtle,None,orig,Immigrants and Refugees,0


In [ ]:
#tokenize text

max_sequence_length = 128

def preprocess_ishate(batch, tokenizer):
    return tokenizer(
        batch["cleaned_text"],
        max_length=max_sequence_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True
    )


In [ ]:
#use roberta this time

!pip install -U transformers


from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "roberta-base"              # or "roberta-large"
roberta_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
roberta_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#tokenize ONCE to speed up training

tokenized_train = ishate_train_data.map(
    preprocess_ishate,
    batched=True,
    fn_kwargs={"tokenizer": roberta_tokenizer},
)
tokenized_val = ishate_val_data.map(
    preprocess_ishate,
    batched=True,
    fn_kwargs={"tokenizer": roberta_tokenizer},
)

Map:   0%|          | 0/55023 [00:00<?, ? examples/s]

Map:   0%|          | 0/4367 [00:00<?, ? examples/s]

In [ ]:
#create evaluation metric
metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:


#taking code from lesson 4 notebook and assignment 2

def fine_tune_classification_model(classification_model,
                                   tokenizer,
                                   train_data,
                                   dev_data,
                                   batch_size = 16,
                                   num_epochs = 2):
    """

    Create the training arguments and trainer for the given model and data.
    Then train it.
    """



    training_args = TrainingArguments(
      output_dir="roberta_ishate",
      per_device_train_batch_size=32,
      per_device_eval_batch_size=32,
      num_train_epochs=num_epochs,
      eval_strategy="epoch",
      save_strategy="no",
      report_to='none',
      fp16=True
    )

    trainer = Trainer(
      model=classification_model,
      args=training_args,
      train_dataset=tokenized_train,
      eval_dataset=tokenized_val,
      compute_metrics=compute_metrics
    )

    trainer.train()

    return trainer

In [ ]:
roBERTa_trainer = fine_tune_classification_model(roberta_model, roberta_tokenizer, ishate_train_data, ishate_val_data)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.194300,0.404121,0.848409
2,0.110800,0.368212,0.861690


In [ ]:
#Tokenize test data with the same preprocessing used for training
#Feed it into trained model
#Convert the model’s output logits into predicted classes
#Calculate accuracy and a detailed classification report showing how well the model distinguishes microaggressions vs. non-microaggressions.

preprocessed_test_data = ishate_test_data.map(preprocess_ishate, batched=True, fn_kwargs={'tokenizer': roberta_tokenizer})
predictions = roBERTa_trainer.predict(preprocessed_test_data)
preprocessed_test_pred = np.argmax(predictions.predictions, axis=1)
from sklearn.metrics import accuracy_score, classification_report

test_accuracy = accuracy_score(y_test_encoded, preprocessed_test_pred)
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test_encoded, preprocessed_test_pred, target_names=label_encoder.classes_))

Map:   0%|          | 0/4368 [00:00<?, ? examples/s]


Test Accuracy: 0.8768

Classification Report:
              precision    recall  f1-score   support

          HS       0.83      0.85      0.84      1687
      Non-HS       0.91      0.89      0.90      2681

    accuracy                           0.88      4368
   macro avg       0.87      0.87      0.87      4368
weighted avg       0.88      0.88      0.88      4368



##Evaluation on Microaggressions Dataset

In [ ]:
#same initial load and format steps as Carlos

micro_agg_url = "https://huggingface.co/spaces/khanak27/microaggressionsdetector/resolve/main/micro_agg.csv"
# Try different encodings to handle Unicode issues
encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1', 'utf-8-sig']

df_micro = None
for encoding in encodings_to_try:
    try:
        print(f"Trying encoding: {encoding}")
        df_micro = pd.read_csv(micro_agg_url, encoding=encoding)
        print(f"✅ Successfully loaded with {encoding} encoding")
        break
    except UnicodeDecodeError as e:
        print(f"❌ Failed with {encoding}: {str(e)[:100]}...")
        continue
    except Exception as e:
        print(f"❌ Other error with {encoding}: {str(e)[:100]}...")
        continue

if df_micro is None:
    print("❌ Failed to load dataset with any encoding. Trying with error handling...")
    try:
        df_micro = pd.read_csv(micro_agg_url, encoding='utf-8', encoding_errors='replace')
        print("✅ Loaded with UTF-8 and error replacement")
    except Exception as e:
        print(f"❌ Final attempt failed: {e}")
        raise

print(f"Dataset shape: {df_micro.shape}")
print(f"Columns: {df_micro.columns.tolist()}")
print(f"\nFirst few rows:")
display(df_micro.head(10))

print(f"\nData types:")
print(df_micro.dtypes)

print(f"\nMissing values:")
print(df_micro.isnull().sum())

print(f"\nLabel distribution:")
print(df_micro['label'].value_counts().sort_index())

# Check for any text preprocessing needed
print(f"\nSample texts:")
for i in range(3):
    print(f"{i+1}. Label {df_micro.iloc[i]['label']}: {df_micro.iloc[i]['speech']}")



df_micro['cleaned_text'] = df_micro['speech']
def map_micro_labels_to_hate_speech(micro_label):
    """Map microaggression labels to hate speech labels"""
    if micro_label == 1:  # Microaggression
        return 'HS'  # Map to hateful speech
    else:  # Normal speech
        return 'Non-HS'  # Map to non-hateful speech

# Apply the mapping
df_micro['mapped_label'] = df_micro['label'].apply(map_micro_labels_to_hate_speech)
y_micro_encoded = label_encoder.transform(df_micro['mapped_label'])

Trying encoding: utf-8
❌ Failed with utf-8: 'utf-8' codec can't decode byte 0xe2 in position 17: invalid continuation byte...
Trying encoding: latin-1
✅ Successfully loaded with latin-1 encoding
Dataset shape: (171, 2)
Columns: ['speech', 'label']

First few rows:


,speech,label
0,You're very articulate for someone like you.,1
1,Where are you really from?,1
2,You're not like other girls.,1
3,You must be good at math since you're Asian.,1
4,You're too pretty to be a software engineer.,1
5,Are you sure you want to lead this project?,1
6,You don't look gay.,1
7,You are a credit to your race.,1
8,"That's a strange name, is it foreign?",1
9,Do you even understand this topic?,1



Data types:
speech    object
label      int64
dtype: object

Missing values:
speech    0
label     0
dtype: int64

Label distribution:
label
0    87
1    84
Name: count, dtype: int64

Sample texts:
1. Label 1: You're very articulate for someone like you.
2. Label 1: Where are you really from?
3. Label 1: You're not like other girls.


In [ ]:
#use modernbert tokenizer

microaggressions_dataset = Dataset.from_dict({
    'cleaned_text': df_micro['cleaned_text'].fillna('').tolist(),
    'label': y_micro_encoded.tolist()
})

preprocessed_microaggressions_data = microaggressions_dataset.map(
    preprocess_ishate,
    batched=True,
    fn_kwargs={'tokenizer': roberta_tokenizer})

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

In [ ]:
#evaluation
micro_predictions = roBERTa_trainer.predict(preprocessed_microaggressions_data)
y_micro_pred = np.argmax(micro_predictions.predictions, axis=1)

micro_accuracy = accuracy_score(y_micro_encoded, y_micro_pred)
print(f"\nMicroaggressions Accuracy: {micro_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(
    y_micro_encoded,
    y_micro_pred,
    target_names=['Normal Speech', 'Microaggression']
))


Microaggressions Accuracy: 0.5614

Classification Report:
                 precision    recall  f1-score   support

  Normal Speech       1.00      0.11      0.19        84
Microaggression       0.54      1.00      0.70        87

       accuracy                           0.56       171
      macro avg       0.77      0.55      0.45       171
   weighted avg       0.76      0.56      0.45       171

